In [1]:
# primeiro passo, devemos utilizar da biblioteca pandas para que
# possamos ler esse csv

import pandas as pd

df = pd.read_csv("reclamacoes_clientes.csv")

df.head(2)


,id,cliente,produto,data,categoria,reclamacao
0,1,Carlos,Tablet A,2024-05-17,Entrega,Comprei o produto há mais de duas semanas e el...
1,2,Mariana,Geladeira Z,2024-05-22,Atendimento,"Realizei o pagamento à vista, via boleto bancá..."


In [2]:
import spacy  # importa a biblioteca spaCy para processamento de linguagem natural

nlp = spacy.load("pt_core_news_sm")  # carrega o modelo de linguagem para português (pequeno)

def preprocess(text):
    if not isinstance(text, str):  # verifica se o input é uma string, senão retorna string vazia
        return ""
    doc = nlp(text.lower())  # converte o texto para minúsculas e processa com spaCy
    # cria uma lista de lemas (forma base das palavras) filtrando tokens que são letras e não são stopwords
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]  
    print(tokens)  # imprime os tokens lematizados para conferência
    return " ".join(tokens)  # retorna os tokens unidos em uma string novamente

# aplica a função preprocess em cada texto da coluna 'resposta_aberta' e cria uma nova coluna 'processed_text'
df["processed_text"] = df["reclamacao"].apply(preprocess)

# imprime as primeiras linhas das colunas originais e processadas para comparar
print(df[["reclamacao", "processed_text"]].head())


['comprei', 'produto', 'haver', 'semana', 'entregar', 'site', 'constar', 'prazo', 'entrega', 'dia', 'útil', 'entrar', 'contato', 'suporte', 'informar', 'produto', 'nenhum', 'previsão', 'claro', 'finalmente', 'recebi', 'aviso', 'entrega', 'pacote', 'chegar', 'danificar', 'embalagem', 'rasgar', 'sinal', 'violação', 'descaso', 'total', 'consumidor', 'enviar', 'cobrar', 'solução', 'ninguém', 'responder', 'sinto', 'totalmente', 'desrespeitar', 'cliente', 'recomendar', 'loja', 'ninguém', 'atendimento', 'extremamente', 'demorar', 'ineficaz', 'solução', 'imediato', 'ter', 'acionar', 'órgão', 'defesa', 'consumidor']
['realizeir', 'pagamento', 'vista', 'via', 'boleto', 'bancário', 'dia', 'compra', 'prazo', 'confirmação', 'dia', 'útil', 'passar', 'dia', 'status', 'pedido', 'alterar', 'entrar', 'contato', 'chat', 'loja', 'recebi', 'resposta', 'automático', 'ligar', 'sac', 'ligação', 'cair', 'várias', 'finalmente', 'conseguir', 'falar', 'atendente', 'dizer', 'ir', 'verificar', 'retornarer', 'contat

text.lower() → tudo em minúsculas ✅

token.is_alpha → remove números, pontuação, emojis etc. ✅

not token.is_stop → remove stopwords ✅

token.lemma_ → mantém o lema da palavra (forma base, como “fazer” em vez de “fazendo”) ✅   

In [3]:
# iremos utilizar os chunks pois os textos estao muito grandes, dessa forma podemos ter ganho em perfomance e economizacao de tokens
from langchain.text_splitter import CharacterTextSplitter

texto_longo = " ".join(df["processed_text"].tolist())

# com o separator consiguimos separar corretamente esses chunks
splitter = CharacterTextSplitter(chunk_size=70, chunk_overlap=20, separator=" ")

chunks = splitter.split_text(texto_longo)

print(chunks)

['comprei produto haver semana entregar site constar prazo entrega dia', 'prazo entrega dia útil entrar contato suporte informar produto nenhum', 'produto nenhum previsão claro finalmente recebi aviso entrega pacote', 'aviso entrega pacote chegar danificar embalagem rasgar sinal violação', 'sinal violação descaso total consumidor enviar cobrar solução ninguém', 'solução ninguém responder sinto totalmente desrespeitar cliente', 'desrespeitar cliente recomendar loja ninguém atendimento extremamente', 'extremamente demorar ineficaz solução imediato ter acionar órgão', 'ter acionar órgão defesa consumidor realizeir pagamento vista via', 'pagamento vista via boleto bancário dia compra prazo confirmação dia', 'confirmação dia útil passar dia status pedido alterar entrar contato', 'entrar contato chat loja recebi resposta automático ligar sac ligação', 'ligar sac ligação cair várias finalmente conseguir falar atendente', 'falar atendente dizer ir verificar retornarer contato acontecer', 'cont

In [4]:
# agora precisamos fazer o embeddamento para que tranformemos em um vetor
# para armazenar em um banco de dados vetorizado

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

print(f"Total chunks:  {len(chunks)}")

embeddings_chunks = model.encode(chunks)

print(f"Gerados {len(embeddings_chunks)} embeddings para chunks, dimensão {embeddings_chunks[0].shape}")

c:\projects\Prova-IA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total chunks:  32
Gerados 32 embeddings para chunks, dimensão (384,)


In [5]:
# agora devemos armazenar em um banco de dados para que consigamos realizar a busca
# semantica

import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings())

collection = client.get_or_create_collection(name="reclamacoes")

ids = [str(i) for i in range(len(chunks))]

metadatas = [{"Source": "reclamacao", "chunk_index": i} for i in range(len(chunks))]

collection.add(
    ids=ids,
    documents=chunks,
    embeddings=embeddings_chunks,
    metadatas=metadatas
)

In [6]:
# agora ja podemos realizar uma consulta nesse nosso banco de dados vetorizado

query_text = "As entregas são feitas dentro do prazo?"

query_embedding = model.encode([query_text][0])

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances", "metadatas"]
)

print("Top 3 resultados mais similares")

for doc, dist, meta in zip(results["documents"][0], results["distances"][0], results["metadatas"][0]):
    print(f"Distância: {dist:.4f} | Meta: {meta} | Texto: {doc}\n")

Top 3 resultados mais similares
Distância: 13.5885 | Meta: {'chunk_index': 14, 'Source': 'reclamacao'} | Texto: contato acontecer produto ficar estoque ser paguei disponível

Distância: 14.1985 | Meta: {'chunk_index': 1, 'Source': 'reclamacao'} | Texto: prazo entrega dia útil entrar contato suporte informar produto nenhum

Distância: 15.2585 | Meta: {'Source': 'reclamacao', 'chunk_index': 0} | Texto: comprei produto haver semana entregar site constar prazo entrega dia

